1. pair representations $h_v^{(0)}$  赋值子图中每个节点和源节点信息
2. Message $M_v^{(1)}$: {$h_v^{(0)}$, 所有指向v的节点x的$h_x$和(x, v)总结后的信息}
3. 总结所有Message

In [2]:
import pandas as pd
sample = pd.read_csv('sample.csv')
cite = pd.read_csv('sample_cite.csv')

# 第一步
src = sample[sample.pid == 144330]
# tar = sample[sample.pid == 63486]
v_df = sample[sample.pid != src.pid.values[0]].reset_index(drop=True)

h_v = pd.DataFrame({
    'src_pid': [src.pid.values[0]] * len(v_df),
    'src_title': [src.title.values[0]] * len(v_df),
    'src_abs': [src['abs'].values[0]] * len(v_df),
    'src_label': [src.label.values[0]] * len(v_df),
    'v_pid': v_df.pid,
    'v_title': v_df.title,
    'v_abs': v_df['abs'],
    'v_label': v_df.label
})

In [3]:
sample

,pid,title,abs,label,h0_10169,h1_10169,h0_144330,h1_144330
0,63486,Title: Evaluation and Selection of Biases in M...,"Abstract: In this introduction, we define the ...",Theory,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","\nThe source paper, ""Utilizing Prior Concepts ...","The source paper, ""Utilizing Prior Concepts fo..."
1,2440,"Title: Quinlan, 1990 J.R. Quinlan. Learning lo...",Abstract: We describe a ranked-model semantics...,Rule_Learning,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","The source paper, ""Utilizing Prior Concepts fo...","The source paper, ""Utilizing Prior Concepts fo..."
2,3231,Title: Irrelevant Features and the Subset Sele...,Abstract: We address the problem of finding a ...,Theory,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","The source paper, ""Utilizing Prior Concepts fo...","The source paper, ""Utilizing Prior Concepts fo..."
3,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,NaN,NaN,"The source paper, ""Utilizing Prior Concepts fo...","The source paper, ""Utilizing Prior Concepts fo..."
4,954315,Title: Inductive Database Design,Abstract: When designing a (deductive) databas...,Rule_Learning,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","\nThe source paper, ""Utilizing Prior Concepts ...","The source paper, ""Utilizing Prior Concepts fo..."
5,144330,Title: Utilizing Prior Concepts for Learning,Abstract: The inductive learning problem consi...,Theory,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...",NaN,NaN
6,56115,Title: Improving Tactical Plans with Genetic A...,Abstract:,Genetic_Algorithms,"The source paper, ""Learning Boolean Concepts ...","The source paper, ""Learning Boolean Concepts i...",It seems there's no abstract provided for the ...,"The source paper, ""Utilizing Prior Concepts fo..."
7,72908,Title: Applications of a logical discovery engine,Abstract: The clausal discovery engine claudie...,Rule_Learning,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","The source paper, ""Utilizing Prior Concepts fo...","The source paper, ""Utilizing Prior Concepts fo..."
8,83461,Title: Dlab: A Declarative Language Bias Forma...,Abstract: We describe the principles and funct...,Rule_Learning,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","The source paper, ""Utilizing Prior Concepts fo...","The source paper, ""Utilizing Prior Concepts fo..."
9,17811,Title: #1 Robust Feature Selection Algorithms,Abstract: Selecting a set of features which is...,Genetic_Algorithms,"The source paper, ""Learning Boolean Concepts i...","The source paper, ""Learning Boolean Concepts i...","The source paper, ""Utilizing Prior Concepts fo...","The source paper, ""Utilizing Prior Concepts fo..."


In [4]:
# Indicator
# h0

for i in range(len(h_v)):

    src_title, src_abs, src_label = h_v.loc[i, 'src_title'], h_v.loc[i, 'src_abs'], h_v.loc[i, 'src_label']
    tar_pid, tar_title, tar_abs, tar_label = h_v.loc[i, 'v_pid'], h_v.loc[i, 'v_title'], h_v.loc[i, 'v_abs'], h_v.loc[i, 'v_label']

    # if tar['pid'].values[0] == tar_pid:
    #     continue
    
    print(tar_pid)

    Indicator_prompt = f"""
Given information of a source paper and a target paper shown as follow:
source paper:
title:{src_title}
abstract:{src_abs}
label:{src_label}
target paper:
title:{tar_title}
abstract:{tar_abs}
label:{tar_label}
please summarize information of these two papers in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
"""

    print(Indicator_prompt)
    print('======================================================================================')


63486

Given information of a source paper and a target paper shown as follow:
source paper:
title:Title: Utilizing Prior Concepts for Learning
abstract:Abstract: The inductive learning problem consists of learning a concept given examples and non-examples of the concept. To perform this learning task, inductive learning algorithms bias their learning method. Here we discuss biasing the learning method to use previously learned concepts from the same domain. These learned concepts highlight useful information for other concepts in the domain. We describe a transference bias and present M-FOCL, a Horn clause relational learning algorithm, that utilizes this bias to learn multiple concepts. We provide preliminary empirical evaluation to show the effects of biasing previous information on noise-free and noisy data.
label:Theory
target paper:
title:Title: Evaluation and Selection of Biases in Machine Learning
abstract:Abstract: In this introduction, we define the term bias as it is used in

In [5]:
# message
# 遍历cite表所有src_id 不是u的边，获得对应的src_id(x)
# 从sample表里获得x的h0，并和cite关系w聚合，得到m
# 将m存入cite表

for i in range(len(cite)):
    src_id, tar_id = cite.loc[i, 'src_id'], cite.loc[i, 'tar_id']
    
    # 隐去所有以src_id为源的边
    if src_id == src.pid.values[0]:
        # print(src_id, tar_id)
        continue
    
    src_title, src_abs, src_label = sample[sample.pid == src_id].title.values[0], sample[sample.pid == src_id]['abs'].values[0], sample[sample.pid == src_id].label.values[0]
    tar_title, tar_abs, tar_label = sample[sample.pid == tar_id].title.values[0], sample[sample.pid == tar_id]['abs'].values[0], sample[sample.pid == tar_id].label.values[0]
    
    # hx
    hx = sample[sample.pid == src_id][f'h0_{src.pid.values[0]}'].values[0]

    w_prompt = f"""title:{src_title}, label:{src_label} cites title:{tar_title}, label:{tar_label}."""

    message_prompt = f"""
Given the source and target papers, and information about the paper cited by target paper:
information of source and target papers: {hx}

the paper which target paper cites: {w_prompt}

please summarize information of source and target papers and the citation relationship in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
"""

    print(src_id, tar_id)
    
    print(message_prompt)

    print('================================================================================')

2440 72908

Given the source and target papers, and information about the paper cited by target paper:
information of source and target papers: The source paper, "Utilizing Prior Concepts for Learning," focuses on leveraging previously learned concepts in inductive learning algorithms, introducing a transference bias via the M-FOCL algorithm for multi-concept learning and presenting empirical evaluations on its impact on noise-free and noisy data. The target paper, "Quinlan, 1990 J.R. Quinlan. Learning logical definitions from relations," introduces a ranked-model semantics for if-then rules with exceptions, emphasizing causal and evidential reasoning, Markov shielding for causation representation, and the resolution of classical issues in prediction, abduction, and belief revision.

Keywords:
Source Paper: Inductive Learning, Transference Bias, M-FOCL Algorithm, Multi-concept Learning, Empirical Evaluation
Target Paper: Ranked-Model Semantics, If-Then Rules, Causal Reasoning, Evidenti

In [6]:
# 遍历每个不是源点的sample
# 找到所有以该点为终点的边，组合message
# 

src_id = src.pid.values[0]

for i in range(len(sample)):
    v_id, h0 = sample.loc[i, 'pid'], sample.loc[i, f'h0_{src_id}']
    m_list = cite[(cite.tar_id == v_id) & (cite.src_id != src_id)][f'm_{src_id}'].values

    # print(v_id, m_list)

    agg_prompt = f'''
given information of a source paper and a target paper shown as follow:
{h0}
'''

    print(v_id)
    if len(m_list) > 0:
        agg_prompt = f'''{agg_prompt}\n
the information of paper which cites the target paper and their relation of the source paper shown as follow:
'''
        for j, m in enumerate(m_list):
            agg_prompt = f"{agg_prompt}\n{j+1}. [{m}]\n"
    agg_prompt = f'''{agg_prompt}
please summarize information in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
'''
    print(agg_prompt)
    print('================================================================================================')


63486

given information of a source paper and a target paper shown as follow:

The source paper, "Utilizing Prior Concepts for Learning," discusses using previously learned concepts to bias the learning method in inductive learning algorithms, showcasing the application of transference bias in the M-FOCL algorithm to learn multiple concepts. It presents empirical evaluation on the impact of leveraging prior information on both noise-free and noisy data. The target paper, "Evaluation and Selection of Biases in Machine Learning," introduces the concept of bias in ML systems, emphasizing the need for automated methods to evaluate and select biases, summarizing recent research in this area.

Keywords:
Source Paper: Utilizing Prior Concepts, Inductive Learning, Transference Bias, M-FOCL Algorithm, Multiple Concepts, Empirical Evaluation
Target Paper: Bias in Machine Learning, Automated Bias Evaluation, Bias Selection, Meta-bias Spaces, Recent Research


the information of paper which cites

In [7]:
# prediction

src_title = sample[sample.pid == src_id]['title'].values[0]


for i in range(len(sample)):
    if sample.loc[i, 'pid'] == src_id:
        continue

    tar_pid, tar_title = sample.loc[i, 'pid'], sample.loc[i, 'title']

    h1 = sample.loc[i, f'h1_{src_id}']
    print(src_id, tar_pid)


    prediction_prompt = f"""
given information of a source paper and a target paper shown as follow:
{h1}
Is there an existing citation relationship between "{src_title}" and "{tar_title}"
Do not give any reasoning or logic for your answer. Answer Yes if they have cited relationship, otherwise, answer No.
answer:

"""
    print(prediction_prompt)
    print('=========================')

144330 63486

given information of a source paper and a target paper shown as follow:
The source paper, "Utilizing Prior Concepts for Learning," explores leveraging prior knowledge through transference bias in inductive learning via the M-FOCL algorithm, assessing its impact on noisy and noise-free data. The target paper, "Evaluation and Selection of Biases in Machine Learning," is cited in various works focusing on theoretical considerations regarding bias assessment in machine learning but lacks specific details in the provided context.

Keywords:
Source Paper: Inductive Learning, Transference Bias, M-FOCL Algorithm, Prior Knowledge, Empirical Evaluation, Noisy Data, Noise-Free Data
Target Paper: Bias Assessment, Theoretical Considerations, Citation, Machine Learning






Is there an existing citation relationship between "Title: Utilizing Prior Concepts for Learning" and "Title: Evaluation and Selection of Biases in Machine Learning"
Do not give any reasoning or logic for your answ